<a href="https://colab.research.google.com/github/zampierilucas/Download-to-Google-Drive/blob/master/Url_download_to_GDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Mount Google Drive

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

###Install dependencies

In [ ]:
!pip install pySmartDL rarfile

###Add URLs e Download

In [ ]:
import requests
import sys
import time
import os
import rarfile
import shutil
import zipfile
from google.colab import drive
from io import StringIO
from pySmartDL import SmartDL


def extract_compressed(source, dest):
  if source.endswith(".zip"):
    with zipfile.ZipFile(source, "r") as zip_ref:
      zip_ref.extractall(path)

  else:
    with rarfile.RarFile(source, "r") as rar_ref:
      rar_ref.extractall(path)

  os.remove(source)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

# WIP
def download_file_from_google_drive(id, destination):    
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

downloads = []
path = "/content/drive/My Drive/Downloads/"

if not os.path.exists(path):
    os.makedirs(path)

while True:
    link = input("Enter URL Or Type Exit: ")
    if link.lower() == "exit":
        break
    downloads.append(link)

sys.stdout.write("Downloading\n")

while downloads:

    file_name = (downloads[0].split("/")[-1]).replace("%20"," ")

    obj = SmartDL(downloads, progress_bar=False)
    obj.start(blocking=False)

    while not obj.isFinished():
      if obj.get_status() == "downloading":
        sys.stdout.write(f"\rDownloading {file_name} {obj.get_progress_bar()} {obj.get_progress()*100:.2f}% {obj.get_dl_size(human=True)} {obj.get_speed(human=True)}   ")
        time.sleep(0.5)

      else:
        sys.stdout.write(f"\rCombining {file_name}  ")
        time.sleep(1)

    if obj.isSuccessful():      
      if file_name.endswith(('.zip', '.rar')):
        sys.stdout.write(f"\rExtracting  {file_name} ")
        extract_compressed(obj.get_dest(), path)
      else:
        destination = path + obj.get_dest().split("/")[-1]
        
        if os.path.exists(destination):
          sys.stdout.write("\rRemoving file with the same name on destination")
          os.remove(destination)
        
        shutil.move(obj.get_dest(),path)
          
    else:
        sys.stdout.write("There were some errors:")
        for e in obj.get_errors():
                sys.stdout.write(str(e))

    sys.stdout.write(f"\r\rCompleted {file_name} File size: {obj.get_dl_size(human=True)} Time taken: {obj.get_dl_time(human=True)}   \n")
    del downloads[0]
    
sys.stdout.write("\nAll downloads have been complete")